# 2. Design principles governing the rate of gene expression

<hr>

**Key concepts and design principles**

- Protein degradation rates determine response times.
- Increased protein turnover speeds up the response time of a gene expression system. 
- Negative autoregulation accelerates turn-on times, but not turn-off times.

**Techniques**

- Steady state normalization allows analysis of response times. 
- Numerical solution of ODEs using Python.
- Interactive plotting using Bokeh.

<hr>


In [1]:
# Colab setup ------------------
import os, sys, subprocess
if "google.colab" in sys.modules:
    cmd = "pip install --upgrade watermark"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
# ------------------------------

import numpy as np
import scipy.integrate
import scipy.optimize

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting

import colorcet

# Set to True to have fully interactive plots
fully_interactive_plots = False
notebook_url = "localhost:8888"

bokeh.io.output_notebook()

Loading BokehJS ...

<hr>

## Protein stability determines the response time to a change in gene expression

So far, we have focused on what happens to a system at steady state. However, biological circuits change dynamically, even in constant environmental conditions. 

The most basic dynamic question one can ask about a gene is how rapidly it can switch from one level of protein to a new level after a sudden increase or decrease in gene expression.

Starting with our basic equation for gene expression, and neglecting the mRNA level for the moment, we write:

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t} = \beta - \gamma x
\end{align}

Consider a situation in which the gene has been off for a long time, so that $x(0)=0$ and then is suddenly turned on at $t=0$. Solving the above differential equation yields the general solution, 

\begin{align}
x(t) = \frac{\beta }{\gamma} (1-e^{-\gamma t}).
\end{align}

Thus, the parameter that determines the response time is $\gamma$, the degradation + dilution rate of the protein. In the above expression, we see that, because $\gamma$ multiplies time, the characteristic response time scale is $\gamma^{-1}$. A plot of the concentration of gene product over time is shown below for $\beta = 100$ and $\gamma = 1$. Note that $\beta$ has dimension of concentration per time and $\gamma$ has dimension of inverse time, and that we take their respective units as arbitrary in the plots that follow.

In [2]:
# Parameters
beta = 100
gamma = 1

# Dynamics
t = np.linspace(0, 6, 400)
x = beta / gamma * (1 - np.exp(-gamma * t))

# Plot response
p = bokeh.plotting.figure(
    frame_height=275,
    frame_width=375,
    x_axis_label="time",
    y_axis_label="x(t)",
    x_range=[0, 6],
)
p.line(t, x, line_width=2)

# Mark the response time (when we get to level 1-1/e)
t0 = 1 / gamma
x0 = beta / gamma * (1 - np.exp(-1))

# Add the glyph with label
source = bokeh.models.ColumnDataSource(
    data=dict(t0=[t0], x0=[x0], text=["response time = 1/γ"])
)
p.circle(x="t0", y="x0", source=source, size=10)
p.add_layout(
    bokeh.models.LabelSet(
        x="t0", y="x0", text="text", source=source, x_offset=10, y_offset=-10
    )
)
p.add_layout(
    bokeh.models.Span(
        location=t0,
        level="underlay",
        dimension="height",
        line_color="black",
        line_dash="dashed",
    )
)

bokeh.io.show(p)

## How can we speed up responses?

So far, it seems as if the cell's ability to modulate the concentration of a stable protein is quite limited, apparently requiring multiple cell cycles for both increases and decreases. This seems rather pathetic for such a successful, billion year old creature. You might think you could up-regulate the protein level faster by cranking up the promoter strength (increasing $\beta$). Indeed, this would allow the cell to hit a specific threshold earlier. However, it would also increase the final steady state level ($\beta/\gamma$), and therefore leave the timescale over which the system reaches its new steady-state unaffected.

One simple and direct way to speed up the response time of the protein is to destabilize it, increasing $\gamma$. This strategy pays the cost of a **futile cycle** of protein synthesis and degradation to provide a benefit in terms of the speed with which the regulatory system can reach a new steady state. 

To disentangle amplitude and timescale, it is useful to normalize these plots by their steady states, as in the bottom plot below. Comparing normalized response curves is reasonable, because mutations can alter the expression level of a gene allowing evolution to re-optimize expression level independently of the regulatory feedback system.

In [3]:
# Parameters
beta_1 = 100
gamma = np.array([1, 2, 3])

# Compute dynamics
t = np.linspace(0, 6, 400)
x = [beta_1 / g * (1 - np.exp(-g * t)) for g in gamma]

# Set up plots
colors = bokeh.palettes.Blues5
p1 = bokeh.plotting.figure(
    frame_height=175,
    frame_width=300,
    x_axis_label="time",
    y_axis_label="x(t)",
    x_range=[0, 6],
)
p2 = bokeh.plotting.figure(
    frame_height=175,
    frame_width=300,
    x_axis_label="time",
    y_axis_label="x(t)",
    x_range=[0, 6],
)
p2.x_range = p1.x_range

# Populate graphs
for x_vals, g, color in zip(x, gamma, colors):
    p1.line(t, x_vals, color=color, line_width=2)
    p1.circle(1 / g, beta_1 / g * (1 - np.exp(-1)), color=color, size=10)
    p2.line(t, x_vals / x_vals.max(), color=color, line_width=2)
    p2.circle(1 / g, 1 - np.exp(-1), color=color, size=10)
    

# Label lines
p1.text(
    x=[4],
    y=[95],
    text=[f"γ = {gamma[0]}"],
    text_color=colors[0],
    text_font_size="10pt",
    text_align="left",
    text_baseline="top",
)
p1.text(
    x=[4],
    y=[53],
    text=[f"γ = {gamma[1]}"],
    text_color=colors[1],
    text_font_size="10pt",
    text_align="left",
    text_baseline="bottom",
)
p1.text(
    x=[4],
    y=[30],
    text=[f"γ = {gamma[2]}"],
    text_color=colors[2],
    text_font_size="10pt",
    text_align="left",
    text_baseline="top",
)

bokeh.io.show(bokeh.layouts.gridplot([p1, p2], ncols=1))

This analysis reveals an important **design principle**: _Increased protein degradation can speed up the response time of a gene expression system, at the cost of additional protein production._

## Network motifs identify functionally important circuits.

We have just seen that destabilizing a protein can speed its response time. However, most bacterial proteins, transcription factors in particular, are stable. Do they use other mechanisms to accelerate response times? 

The answer to this question will turn out to be yes, there is a circuit that can  accelerate responses. But before we get there, let's step back for a moment to ask how one can go about discovering such functionally important circuits in the first place. Do you have to guess them? It would be nice if there were some kind of catalog of important circuits and their functions that we could browse.

The concept of **network motifs** is one way to obtain such a catalog. We define a network motif as a regulatory pattern, or sub-circuit, that is statistically over-represented in natural networks (circuits), compared to what one might expect from random networks with similar numbers of genes and regulatory interactions. 

More specifically, imagine the transcriptional regulatory network of an organism as a **graph** consisting of **nodes** and **directed edges** (arrows). In bacteria, each node represents an operon, while each arrow represents regulation of the target operon (tip of the arrow) by a transcription factor in the originating operon (base of the arrow), as shown schematically here.

<div style="width: 300px; margin: auto;">

![simple graph](figs/simple_graph_2.png)

</div>

The transcriptional regulatory network of _E. coli_ has been mapped (see [RegulonDB](http://regulondb.ccg.unam.mx)). It contains ≈424 operons (nodes), ≈519 transcriptional regulatory interactions (arrows), involving ≈116 transcription factors. If the target of each arrow was chosen randomly, the probability of any given arrow being autoregulatory is low (≈1/424). One might expect only about one such event in the entire network. However, ≈40 such autoregulatory arrows are observed. Autoregulation thus appears to be statistically over-represented compared to the null hypothesis of random regulatory interactions. 

(If we further consider the "sign" of the arrow, with "+" representing activation and "-" representing repression, it turns out that there are 32 negative autoregulatory operons and 8 positive autoregulatory ones. We will discuss both types.)

The **motif principle** states that statistically over-represented patterns in networks have been selected repeatedly because they provide key cellular functions. A similar concept is useful in other aspects of biology. For example, sequence motifs are statistically over-represented sequences within the genome that are enriched for functionally important features, such as protein binding sites. This is a core concept in bioinformatics. 

## Negative autoregulation accelerates response times

Identifying motifs in regulatory networks is one way to discover functionally important circuits, and one of the strongest motifs of them all is autoregulation. You can guess that since we are discussing speeding up circuit response that it does just that. So, how does it work? 

To start, we write down a differential equation representing production and degradation of the repressor, $x$. We represent its regulation using a Hill repression function, and we keep the Hill coefficient at $n=1$ for now.

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t}=\frac{\beta}{1+x/k} - \gamma x
\end{align}

We will consider the limit in which the autoregulation is "strong", i.e. where $\beta/\gamma \gg k$, so the gene can, at maximal expression level, produce enough protein to fully repress itself. 

What happens when the operon is suddenly turned "on" from an initial "off" state ($x(0)=0$)? Initially, $x$ builds up approximately linearly, at rate $\beta$. Eventually, its concentration is high enough to shut its own production off when  $x \approx k$. While the real dynamics are smoother, one can think of the behavior roughly as in the plot below.

In [4]:
# Curve
t = [0, 1, 2]
x = [0, 1, 1]

# Set up plot
p = bokeh.plotting.figure(
    frame_height=175,
    frame_width=300,
    x_axis_label="time",
    y_axis_label="x(t)",
    x_range=[0, 2],
)

# Custom axis labels
p.xaxis.ticker = bokeh.models.tickers.FixedTicker(ticks=[0, 0.5, 1])
p.yaxis.ticker = bokeh.models.tickers.FixedTicker(ticks=[0, 1])
p.xaxis.major_label_overrides = {0.5: "$$t_{1/2}$$", 1: "$$t_0$$"}
p.yaxis.major_label_overrides = {1: "$$k$$"}

# Populate plot
p.line(t, x, line_width=2)

# Label slope
p.line([0.2, 0.2, 0.3], [0.2, 0.3, 0.3], color='black')
p.text(
    x=[0.125],
    y=[0.25],
    text=["β"],
    text_color="black",
    text_font_size="12pt",
)

bokeh.io.show(p)

In this sketch, we can see that the half-time, $t_{1/2}$ for turning on should occur when $\beta t \approx k / 2$, i.e. at $t \approx k/2 \beta$. But this is only an approximation. A more complete treatment in <a href="https://doi.org/10.1016/S0022-2836(02)00994-4">Rosenfeld et al., JMB 2002</a> shows that in the limit of strong negative autoregulation the dynamics approach

\begin{align}
x(t) \approx x_\mathrm{ss} \sqrt{1-\mathrm{e}^{-2 \gamma t}},
\end{align}

where $x_\mathrm{ss}$ denotes the steady-state expression level. In the following sections, we will explore these dynamics using numerical integration and compare them to this analytical approximation.

### Incorporating explicit input dependence into a model of regulation

Throughout this chapter we have been invoking a situation where the gene has been off for some time before it is suddenly turned on. We previously saw that it is possible for an inducer molecule to inhibit the action of a repressor, but it is also possible for there to be molecules that can activate the action of a repressor, or inhibit the action of an activator, or many other types of interactions. For now, we will ignore specific biomolecular details and simply write out a generic dependence of the autorepressive gene's expression on the presence of some external signal, $s$.

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t} = \frac{\beta(s)}{1 + (x/k)^n} - \gamma x
\end{align}

Note that we have explicitly included the Hill coefficient $n$ to allow for ultrasensitivity. We we will explore its effects on the dynamics later.

We are now faced with a question: What is the functional form of $\beta(s)$? There are some properties of the relationship between $\beta$ and $s$ that we will always want to include, such as the fact that $\beta$ should increase with $s$ and that $\beta$ should have a value close to $0$ when $s$ is close to $0$. Furthermore, there should be a limit to how rapidly the gene product can be produced, so $\beta(s)$ must be finite, even for large $s$. As we saw in the previous chapter, the Hill function satisfies these properties and provides, and we can use it to phenomenologically model activation by $s$. The two parameters of a Hill function, $k$ and $n$, also offer flexibility in the shapes of the phenomenological curves. As a result, Hill functions are effectively used in models for many different biological systems. Here, we will represent $\beta(s)$ as an activating Hill function

\begin{align}
\beta (s) = \beta_0\, \frac{(s/k_s)^{n_s}}{1 + (s/k_s)^{n_s}}.
\end{align}

We could choose $n_s = 1$, which would give a linear relationship between $\beta$ and $s$ in the regime where $s \ll k_s$. Alternatively, we could get a step-like response in $\beta$ at $s = k_s$ if we choose $n_s$ to be very large. It is important to understand that here that the $\beta(s)$ Hill function is not representing a process of transcriptional regulation like the Hill term in our original $\mathrm{d}x/\mathrm{d}t$ expression, but is instead phenomenologically representing a process of enzymatic inactivation of the repressor $x$ by the external signal $s$.

Having chosen to represent $\beta(s)$ as an activating Hill function, our ODE model for our autoregulated gene now takes the form

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t} = \beta_0\,\frac{(s/k_s)^{n_s}}{1 + (s/k_s)^{n_s}}\, \frac{1}{(1 + (x/k)^n)} - \gamma x.
\end{align}

### The scipy.intergrate module

Now that we have our dynamical equations, we will move to solving them.

**The [SciPy Library](https://docs.scipy.org/doc/scipy/reference/)** is a Python library for scientific computing. It contains many modules, including `scipy.stats`, `scipy.special`, and `scipy.optimize`, which respectively have functions to perform statistical calculations, special functions, and optimization routines. There are many more. We will use the `scipy.integrate` module to integrate systems of ODEs.

There are three main APIs for solving real-valued initial value problems in the module. They are [solve_ivp()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html#scipy.integrate.solve_ivp), [ode()](https://scipy.github.io/devdocs/generated/scipy.integrate.ode.html#scipy.integrate.ode), and [odeint()](https://scipy.github.io/devdocs/generated/scipy.integrate.odeint.html#scipy.integrate.odeint). According to the SciPy developers, `solve_ivp()` is the preferred method, with the others labeled as having an "old" API. The `solve_ivp()` function has the flexibility of allowing choice of multiple numerical algorithms for solving ODEs. However, for the kinds of problems we encounter in this class, I find that the generic LSODA algorithm developed by Linda Petzold and Alan Hindmarsh that handles both stiff and non-stiff problems with variable time stepping is the best option. This is the only solver offered in the `odeint()` function. If we compare the two solvers, `solve_ivp()` and `odeint()`, the former has a large overhead, which can lead to performance issues for small problems (for large problems, the overhead is negligible). Since most of our problems are small and we need to solve them rapidly in interactive graphics, we will use `odeint()`. Because it lack the overhead, for these kinds of applications it has much better performance, and though its API is different, it is still intuitive.

The basic call signature for `odeint()` is

    scipy.integrate.odeint(func, y0, t, args=())

There are many other keyword arguments to set algorithmic parameters, but we will generally not need them (and you can read about them in the [documentation](https://scipy.github.io/devdocs/generated/scipy.integrate.odeint.html#scipy.integrate.odeint)). Importantly, `func` is a vector-valued function with call signature `func(y, t, *args)` that specifies the right hand side of the system of ODEs to be solved. `t` is a scalar time point and `y` is a one-dimensional array (though multidimensional arrays are possible). `y0` is an array with the initial conditions.

As is often the case, use of this function is best seen by example, and we will now apply it to the negative autoregulation circuit.

### Solving for a constant input

We will first consider the case where we initially have no $s$ present, but a step increase in $s$ at time $t=0$ causes a corresponding step-like increase in the value of $\beta$, which we represent by having $n_s \gg 1$ and $s \gg k_s$.

If we are only simulating our system starting at $t=0$, then we can treat $s$ as a single, constant parameter, rather than a time-varying variable. So, we need seven parameters for the right hand side of our ODEs: $\beta_0$, $\gamma$, $k$, $n$, $k_s$, $n_s$, and $s$.

We now define the function for the right hand side of the ODEs.

In [5]:
def neg_auto_rhs(x, t, beta0, gamma, k, n, ks, ns, s):
    """
    Right hand side for negative autoregulation motif with s dependence.
    Return dx/dt.
    """
    # Compute dx/dt
    dx_dt = (
        beta0 * (s / ks) ** ns / (1 + (s / ks) ** ns) / (1 + (x / k) ** n) - gamma * x
    )

    # Return the result
    return dx_dt

We can now define the initial conditions, our parameters (taking $n_s = 10$), and the time points we want and use `scipy.integrate.odeint()` to solve.

In [6]:
# Time points we want for the solution
t = np.linspace(0, 10, 200)

# Initial condition
x0 = 0.0

# Parameters
beta0 = 100
gamma = 1
k = 0.5
n = 1
s = 100
ns = 10
ks = 0.1

# Package parameters into a tuple
args = (beta0, gamma, k, n, ks, ns, s)

# Integrate ODES
x = scipy.integrate.odeint(neg_auto_rhs, x0, t, args=args)

That's it! The integration is done. Let's take a quick look at the output.

In [7]:
x.shape

(200, 1)

We see that the output of `odeint()` has each column corresponding to a given species and each row to a given time point. It is often convenient to transpose the output so that the each species is more easily index-able.

In [8]:
# Extract time course for first (in this case only) species
x = x.transpose()[0]

We we plot the result, we would like to compare it both to the limiting result of <a href="https://doi.org/10.1016/S0022-2836(02)00994-4">Rosenfeld et al.</a>,

\begin{align}
x(t) \approx x_\mathrm{ss} \sqrt{1-\mathrm{e}^{-2 \gamma t}},
\end{align}

and also to the unregulated case,

\begin{align}
x(t) = \frac{\beta_0}{\gamma}\left(1 - \mathrm{e}^{-\gamma t}\right).
\end{align}

In [9]:
# Unregulated solution
x_unreg = beta0 / gamma * (1 - np.exp(-gamma * t))

# Limiting analytical solution
x_limiting = x[-1] * np.sqrt(1 - np.exp(-2 * gamma * t))

Now let's now plot the results.

### Plotting results

Like all of the plots in this book, we use [Bokeh](https://docs.bokeh.org/) to generate the plot of our results. Bokeh is an excellent plotting tool that affords interactivity in the browser, and we use it to great effect in this and most other chapters.

The syntax is mostly self-explanatory from the example below. Note that you can save a plot as a PNG by clicking the disk icon next to the plot, which might be helpful for incorporating your plots into your presentations. (Note that for publications, you should usually save your figures in a vector graphics format, which [Bokeh supports](https://docs.bokeh.org/en/latest/docs/user_guide/export.html), but is not necessary, and in fact discouraged, for display of plots in the browser.) 

Before building the plot, we will load in the color scheme we will use. The [colorcet package](https://colorcet.holoviz.org/) is good for this. We prefer the Category10 color palette for categorical colors.

In [10]:
# Set up color palette for this notebook
colors = colorcet.b_glasbey_category10

Colors in place, we first set up a figure on which to place the plot.

In [11]:
# Set up figure
p = bokeh.plotting.figure(
    frame_width=325,
    frame_height=250,
    x_axis_label="time",
    y_axis_label="x",
    x_range=[np.min(t), np.max(t)],
)

Next, we populate the plot with the curves we want to show, the numerical solution, the Rosenfeld limiting solution, and the unregulated. We could make a plot with a function call like this:

```python
p.line(t, x, line_width=2, color=colors[0], legend_label="numerical solution")
```

The method `p.line()` populates a figure `p` with a line with x-values given by the first argument and y-values given by the second argument. This method is a useful shortcut for quickly making plots, but it is useful to instead define a **ColumnDataSource** that contains the data for a plot (in this case the points along the curves we are plotting). This allows for the plotted data to be adjusted and the plot updated without generating a new plot.

We make a ColumnDataSource by instantiating it with a dictionary of arrays we wish to use in the plot.

In [12]:
cds = bokeh.models.ColumnDataSource(
    dict(t=t, x=x, x_limiting=x_limiting, x_unreg=x_unreg)
)

Now that we have the ColumnDataSource, we can add the curves to the plot. When we populate the plot with glyphs, we use the `source` keyword argument to specify the `ColumnDataSource`, and then specify `x` and `y` as strings indicating which columns of the `ColumnDataSource` are used to specify the $x$ and $y$ values, respectively.

In [13]:
# Populate glyphs
p.line(source=cds, x="t", y="x", line_width=2, color=colors[0], legend_label="numerical solution")
p.line(source=cds, x="t", y="x_limiting", line_width=2, color=colors[1], legend_label="Rosenfeld limiting solution")
p.line(source=cds, x="t", y="x_unreg", line_width=2, color=colors[2], legend_label="unregulated");

We can tweak the properties of the plot by placing the legend, allowing curves to be shown or hidden by clicking them in the legend, and by adding a title.

In [14]:
# Place the legend
p.legend.location = "center_right"

# Allow hiding some plots
p.legend.click_policy = "hide"

# Write the title
p.title.text = "Constant-input dynamics"

Finally, we show the plot using `bokeh.io.show()` to display the graphic in the notebook. Note that at the top of the notebook, we called `bokeh.io.output_notebook()` which tells `bokeh.io.show()` to display the plot in the notebook instead of writing it out to a file.

In [15]:
bokeh.io.show(p)

The numerical solution and the Rosenfeld limiting solution are nearly identical. To see the numerical solution, you may even need to click the legend to hide the limiting solution.

Since we want to compare the speed with which the system approaches steady state, we should instead plot the normalized expression level. To do that, we can update the data in the ColumnDataSource to contain normalized levels.

In [16]:
cds.data["x"] /= cds.data["x"].max()
cds.data["x_limiting"] /= cds.data["x_limiting"].max()
cds.data["x_unreg"] /= cds.data["x_unreg"].max()

We can complete our updated normalized plot by tweaking the y-axis label.

In [17]:
# Update axis label
p.yaxis.axis_label = "normalized x"

# Show plot
bokeh.io.show(p)

### Design principle: Negative autoregulation of a transcription factor accelerates its response to a change in input

Taken together, the unnormalized and normalized plots expose two effects of the autoregulation. First, as you would expect, adding negative autoregulation reduces the steady-state expression level. As a second effect, the approach to steady state is accelerated. This can be seen in the second plot where all the concentrations are normalized to their steady state. In fact, negative autoregulation has accelerated the dynamics by about 5-fold compared to the unregulated system. We have thus arrived at another **design principle**: _Negative autoregulation of a transcription factor accelerates its response to a change in input_.

Note that this acceleration occurs when we turn the gene on, but one might ask whether such acceleration also holds when we turn the gene off, for example by suddenly stopping the gene's expression at some later time point. We can explore this question and others by including $s$ as a time-varying signal into our ODE solver.

## Solving for a time-varying input

Now let's imagine that the input signal $s$, rather than turning on instantaneously at $t=0$ and remaining constant for the rest of the simulation, instead occurs as a pulse of duration $\tau$ that peaks at time $t_0$. Then $s$ would be present roughly within the time window $(t_0-\tau/2, t_0+\tau/2)$, but absent outside of this time window. We will model the peak as a Gaussian function of unit height,

\begin{align}
s(t) = \exp\left[-\frac{4(t-t_0)^2}{\tau^2}\right].
\end{align}

In order to incorporate these dynamics into our model, we can write a function.

In [18]:
def s_pulse(t, t_0, tau):
    """
    Returns s value for a pulse centered at t_0 with duration tau.
    """
    # Return 0 is tau is zero, otherwise Gaussian
    return 0 if tau == 0 else np.exp(-4 * (t - t_0) ** 2 / tau ** 2)

Let's take a look at what the pulse looks like for visualization purposes. This is an example where a quick plot suffices; there is no need to set up a ColumnDataSource because we will not be changing the data on this plot.

In [19]:
# Plot the pulse
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="time",
    y_axis_label="s",
    x_range=[0, 10],
)

# Populate glyph
p.line(t, s_pulse(t, 4.0, 2.0), line_width=2)

# Show plot
bokeh.io.show(p)

The input time course has the desired pulsatile shape.

If we want to solve the ODEs for a varying input, we need to have a way to pass a function defining the variation as a parameter. Fortunately, we can pass functions as arguments in Python. So, we write a new function for the right-hand-side of our ODE that takes `s_fun`, the function describing $s(t)$ as an argument, as well as `s_args`, the set of parameters passed into `s_fun`.

In [20]:
def neg_auto_rhs_s_fun(x, t, beta0, gamma, k, n, ks, ns, s_fun, s_args):
    """
    Right hand side for negative autoregulation function, with s variable.
    Returns dx/dt.
    
    s_fun is a function of the form s_fun(t, *s_args), so s_args is a tuple
    containing the arguments to pass to s_fun.
    """
    # Compute s
    s = s_fun(t, *s_args)

    # Plug in this value of s to the RHS of the negative autoregulation model
    return neg_auto_rhs(x, t, beta0, gamma, k, n, ks, ns, s)

Now that we have this new function in hand, we can numerically integrate our ODEs as we did before. We'll start with a pulse that is on from roughly $t=2$ to $t=6$, as above.

In [21]:
# Set up parameters for the pulse
s_args = (4.0, 2.0)

# Package parameters into a tuple
args = (beta0, gamma, k, n, ks, ns, s_pulse, s_args)

# Integrate ODEs
x = scipy.integrate.odeint(neg_auto_rhs_s_fun, x0, t, args=args).transpose()[0]

# Plot the normalized values
x /= x.max()

# Also calculate the pulse for plotting purposes
s = s_pulse(t, *s_args)

# Plot the results
p = bokeh.plotting.figure(
    frame_width=450,
    frame_height=250,
    x_axis_label="time",
    y_axis_label="normalized concentration",
    x_range=[0, 10],
)

# Populate glyphs
p.line(t, s, line_width=2, color=colors[0], legend_label="s")
p.line(t, x, line_width=2, color=colors[1], legend_label="x")

# Place the legend
p.legend.location = "top_right"

# Show plot
bokeh.io.show(p)

As expected, we see the qualitative result that $x$ does not turn on until the pulse of $s$ begins, and once $s$ stops, $x$ begins an exponential decay back down to zero. The specific quantitative details of the relationship between the $x$ trajectory and the $s$ pulse, such as the threshold value of $s$ where $x$ begins to turn on, are dependent on the specific parametrization of the system, such as the value of $k_s$.

We now want to ask how the dynamics of $x$'s rise and fall compare to the unregulated case. In order to do this, we simply need to write another function that plots out the unregulated time course as an explicit function of $s$.

The attentive reader might, at this point, raise an objection: How can we claim that the *unregulated* gene $x$ is nonetheless still governed by the presence or absence of this input signal $s$? Indeed, although an unregulated gene does not have any transcriptional activators or repressors that modulate its expression, all genes in an organism can always have their expression level be affected by higher-level factors such as the concentration of RNA polymerase or ribosomes, the global metabolic rate, or in the case of eukaryotic organisms, chromatin accessibility. So in this case we could imagine that our $s$ pulse represents a very simplified model of a scenario where a bacterium containing our gene is persisting in a nutrient-starved environment, where most of its genes have globally been turned off, and suddenly comes across a pulse of nutrients that allow it to transiently ramp up its metabolism and gene expression before returning to a starvation state when the nutrients disappear.

Let's now put together a function that takes our unregulated ODE and converts it to a form that includes an explicit Hill-like $s$-dependence for the activation term, as in

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t} = \beta(s) - \gamma x = \beta_0\, \frac{(s/k_s)^{n_s}}{1 + (s/k_s)^{n_s}} - \gamma x.
\end{align}

In [22]:
def unreg_rhs(x, t, beta0, gamma, ks, ns, s):
    """
    Right hand side for constitutive gene expression
    modulated to only be active in the presence of s.
    Returns dx/dt.
    """
    return beta0 * (s / ks) ** ns / (1 + (s / ks) ** ns) - gamma * x


def unreg_rhs_s_fun(x, t, beta0, gamma, ks, ns, s_fun, s_args):
    """
    Right hand side for unregulated function, with s variable.
    Returns dx/dt.
    
    s_fun is a function of the form s_fun(t, *s_args), so s_args is a tuple
    containing the arguments to pass to s_fun.
    """
    # Compute s
    s = s_fun(t, *s_args)

    # Plug in this value of s to the RHS of the negative autoregulation model
    return unreg_rhs(x, t, beta0, gamma, ks, ns, s)

We can now numerically solve the unregulated ODE and add it to the plot.

In [23]:
# Package parameters into a tuple
args_unreg = (beta0, gamma, ks, ns, s_pulse, s_args)

# Integrate ODEs
x_unreg = scipy.integrate.odeint(unreg_rhs_s_fun, x0, t, args=args_unreg).transpose()[0]

# Normalize
x_unreg /= x_unreg.max()

# Add to the plot
p.line(t, x_unreg, line_width=2, color=colors[2], legend_label="unregulated x")

# Show plot
bokeh.io.show(p)

We can see from the above plot that although negative autoregulation speeds up the rise time of the gene's expression in response to the appearance of stimulus, it has no impact on the speed of the fall time in response to the disappearance of the stimulus. We actually could have known this result would occur just from looking at the equations governing the gene's dynamics: recall that in the regulated case, $x$ is governed by

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t}=\frac{\beta(s)}{(1+x/k)^n} - \gamma x,
\end{align}

while in the unregulated case, $x$ is governed by

\begin{align}
\frac{\mathrm{d}x}{\mathrm{d}t}=\beta(s) - \gamma x.
\end{align}

If we set $\beta(s)=0$, then both ODEs become the same expression,

\begin{align}
\left.\frac{\mathrm{d}x}{\mathrm{d}t}\right\vert_{\beta(s)=0}=- \gamma x,
\end{align}

meaning that the dynamics of the fall in gene expression will become identical between the two systems as $\beta(s)$ approaches $0$.

Although we have now convincingly demonstrated that it is possible for negative autoregulation to speed the response time of a gene's expression, we don't yet have a clear understanding of how the values of the parameters themselves, particularly $k$ and $n$, will affect the magnitude of this speed-up. In order to help gain some insight into these effects, we will turn to the power of interactive plotting.

## Interactive plotting and varying parameters

Plotting with Bokeh in Jupyter notebooks allows for interactivity with plots that can help to rapidly gain insights about how parameter values might affect the dynamics. We have found that this is a useful tool to rapidly explore parameter dependence on circuit behavior.

To make an interactive plot in Bokeh, there are three major components.

1. The plot or plots themselves.
2. The **widgets**. Widgets for parameter values are primarily sliders, which enable you to vary parameter values by clicking and dragging. We will also make use of other widgets such as toggle, radio buttons, and drop menus throughout the book.
3. The **callback function**. This is a function that is executed whenever a widget changes value. Most of the time, we use it to update a ColumnDataSource of a plot. You may have more than one callback functions for different widgets and also for changes in the range of the axis of the plot due to zooming.
4. The **layout**. This is the spatial arrangement of the plots and widgets.
5. The **app**. Bokeh will create an application that can be embedded in a notebook or serves as its own page in a browser. To create, it you need to make a simple function that adds the layout you built to the document that Bokeh will make into an app. (This sounds a lot more complicated than it is; see the example below.)

We refer to a plot or set of plots with widgets for interactivity as a **dashboard**.

<div class="alert alert-info">

Note 

The excellent package [Panel](https://panel.holoviz.org) allows for more declarative (and hence fewer lines of code) means of dashboarding (with dashboards ultimately being rendered, if desired, with Bokeh) and we encourage you to explore it. We however choose to use base Bokeh for interactive plotting in this book for a few reasons because it offers greater flexibility and performance, allows for convenient JavaScript integration (which is necessary for many of the interactive plots to work in the static HTML rendering of this book), and it is not *that* much more effort to code up dashboards in base Bokeh than in Panel.
 
</div>

### A simple example: Varying the properties of the input signal

To demonstrate dashboard construction, we will first build a simple example that lets you interactively change the properties of the input signal's time course, without incorporating it into the ODE model just yet.

#### Step 1: Generate the plot (but don't show it)

First, we will generate the plot. When we generate the plot, we specify the data in a `ColumnDataSource` so that we can change the data in the plot without re-rendering it.

We cannot show the plot here, since we will show it in the app. A Bokeh plot can only be in a single document, in our case the app (which as far as Bokeh is concerned is separate from showing it in the JupyterLab cell below).

In [24]:
# t/s data for plotting
t_0 = 4.0
tau = 2.0
t = np.linspace(0, 10, 200)
s = np.exp(-4 * (t - t_0) ** 2 / tau ** 2)

# Place the data in a ColumnDataSource
cds = bokeh.models.ColumnDataSource(dict(t=t, s=s))

# Build the plot
p = bokeh.plotting.figure(
    frame_height=200,
    frame_width=400,
    x_axis_label="time",
    y_axis_label="input signal",
    x_range=[0, 10],
    y_range=[0, 1.1],
)
p.line(source=cds, x="t", y="s", line_width=2);

#### Step 2: Make the widgets

Now we will make our widget, in this case two sliders to represent the values $t_0$ and $\tau$ parameters.

In [25]:
t0_slider = bokeh.models.Slider(
    title="t0", start=0, end=10, step=0.01, value=4.0, width=150
)
tau_slider = bokeh.models.Slider(
    title="tau", start=0, end=10, step=0.01, value=2.0, width=150
)

The sliders are instantiated using `bokeh.models.Slider()`, with keyword arguments whose meaning should be obvious from their names. The `value` attribute of the slider is the present value of the slider.

#### Step 3: Make the callbacks

Next, we specify the callback function that will be used to update the plot as the slider changes. The callback function for a slider must take three arguments, the attribute that changes, its old value, and its new one. We will not directly use these arguments (though we could), but will rather directly read the value from the slider using its `value` attribute. Our callback function simply updates the `y` data values of the `ColumnDataSource`.

In [26]:
def callback(attr, old, new):
    cds.data["s"] = s_pulse(cds.data["t"], t0_slider.value, tau_slider.value)

Now, we need to alert Bokeh that it should trigger the callback function whenever the slider `value` changes.

<div class="alert alert-warning">

Warning 

In the static HTML rendering of this notebook, the app will not be responsive. This is because the app needs a running Python engine for the interactivity, and no such engine is available in the browser. Also, Bokeh apps currently are not supported by Google Colab.
    
Therefore, in all chapters with Bokeh apps requiring Python, we have a variable `fully_interactive_plots`, which is set in the first code cell along with the imports. This is set to `False` for static HTML rendering. In this case we do not link the sliders to callbacks (since there is no Python engine), and disable the slider.

In a later chapter, we will show how you can get interactivity in an HTML rendering of a notebook using JavaScript.
       
</div>

In [27]:
if fully_interactive_plots:
    t0_slider.on_change("value", callback)
    tau_slider.on_change("value", callback)
else:
    t0_slider.disabled = True
    tau_slider.disabled = True

#### Step 4: Build the layout

Now that we have a slider and a plot and have linked the data source of the plot to the slider, we can lay out the dashboard. We will put the sliders in a column next to the plot, putting a spacer in between.

In [28]:
layout = bokeh.layouts.row(
    p, bokeh.models.Spacer(width=30), bokeh.layouts.column(t0_slider, tau_slider)
)

#### Step 5: Make the app

Now we are ready to make a function to produce the app. The function needs to have call signature `app(doc)`, where `doc` represents the document that Bokeh will build into an app. The purpose of this function is to add the layout we have just build to the document.

In [29]:
def app(doc):
    doc.add_root(layout)

#### Step 6: Enjoy your interactive plot!

And we are finally ready to see the app! When we call `bokeh.io.show()`, we pass the `app()` function as the first argument. We also need to use the `notebook_url` keyword argument to specify where the notebook is being hosted. This is usually `"localhost:8888"`, but the number may change (e.g., `8889` or `8890`). You can look in the navigation bar of your browser to make sure you get the right number. In this and all other chapters, we specify `notebook_url` in the first cell of the notebook along with the imports.

<div class="alert alert-info">

Note 

In the static HTML rendering of this notebook, the app will not appear. This is because it is running a Python engine for the interactivity, and no such engine is available in the browser. In a later chapter, we will show how you can get interactivity in an HTML rendering of a notebook using JavaScript.
    
Also, Bokeh apps currently are not supported by Google Colab.
       
</div>

In [31]:
if fully_interactive_plots:
    bokeh.io.show(app, notebook_url=notebook_url)
else:
    bokeh.io.show(layout)

#### Serving an app

If you wanted to have a stand-alone interactive plot on its own browser tab, you can put all of the code necessary to generate it in a single `.py` file and then serve it from the command line. For this example, we could have a file `pulse_signal.py` with the following contents.

```python
import numpy as np
import bokeh.plotting
import bokeh.models


def s_pulse(t, t_0, tau):
    """
    Returns s value for a pulse centered at t_0 with duration tau.
    """
    # Return 0 is tau is zero, otherwise Gaussian
    return 0 if tau == 0 else np.exp(-4 * (t - t_0) ** 2 / tau ** 2)


# t/s data for plotting
t_0 = 4.0
tau = 2.0
t = np.linspace(0, 10, 200)
s = np.exp(-4 * (t - t_0) ** 2 / tau ** 2)

# Place the data in a ColumnDataSource
cds = bokeh.models.ColumnDataSource(dict(t=t, s=s))

# Build the plot
p = bokeh.plotting.figure(
    frame_height=200,
    frame_width=400,
    x_axis_label="time",
    y_axis_label="input signal",
    x_range=[0, 10],
    y_range=[0, 1.1],
)
p.line(source=cds, x="t", y="s", line_width=2)

t0_slider = bokeh.models.Slider(
    title="t0", start=0, end=10, step=0.01, value=4.0, width=150
)
tau_slider = bokeh.models.Slider(
    title="tau", start=0, end=10, step=0.01, value=2.0, width=150
)

def callback(attr, old, new):
    cds.data["s"] = s_pulse(cds.data["t"], t0_slider.value, tau_slider.value)

t0_slider.on_change("value", callback)
tau_slider.on_change("value", callback)

layout = bokeh.layouts.row(
    p, bokeh.models.Spacer(width=30), bokeh.layouts.column(t0_slider, tau_slider)
)

def app(doc):
    doc.add_root(layout)
    
# Build the app in the current doc
app(bokeh.plotting.curdoc())
```

After saving that file, you can serve it be doing the following on the command line.

```bash
bokeh serve --show s_pulse.py
```

### An app for the negative autoregulation model

Now that we have gained some familiarity with interactive plotting via Bokeh, we will make a dashboard to allow us to interactively explore the negative autoregulation model. To do so, we will incorporate scipy's ODE integration into the plotting function itself, and we will also demonstrate how to create a button that can toggle a categorical property (like normalization of the results).

#### Step 1: Build the plot

We will build the plot as before. We will initially have it contain the parameters we have been using.

In [32]:
# Integrate ODE
x = scipy.integrate.odeint(neg_auto_rhs_s_fun, x0, t, args=args)
x = x.transpose()[0]
x_unreg = scipy.integrate.odeint(unreg_rhs_s_fun, x0, t, args=args_unreg)
x_unreg = x_unreg.transpose()[0]

# also calculate the input
s = s_pulse(t, *s_args)

# Normalize time courses
x /= x.max()
x_unreg /= x_unreg.max()

# set up the column data source
cds = bokeh.models.ColumnDataSource(dict(t=t, x=x, s=s, x_unreg=x_unreg))

# set up plot
p = bokeh.plotting.figure(
    frame_width=375,
    frame_height=250,
    x_axis_label="time",
    y_axis_label="normalized concentration",
    x_range=[t.min(), t.max()],
)

# Populate glyphs
p.line(source=cds, x="t", y="x", line_width=2, color=colors[1], legend_label="x neg. auto.")
p.line(source=cds, x="t", y="x_unreg", line_width=2, color=colors[2], legend_label="x unreg.")
p.line(source=cds, x="t", y="s", line_width=2, color=colors[0], legend_label="s")

# Place the legend
p.legend.location = "top_left"

#### Step 2: Build the widgets

Now we will build our sliders, one for each parameter. We are interested in varying the parameters $\beta$, $\gamma$, and $x_0$ on a logarithmic scale, so we will set up the sliders to specify $\log_{10} \beta$, $\log_{10}\gamma$ and $\log_{10} x_0$.

In [33]:
log_beta0_slider = bokeh.models.Slider(
    title="log₁₀ β₀", start=-1, end=2, step=0.1, value=np.log10(beta0), width=150
)
log_gamma_slider = bokeh.models.Slider(
    title="log₁₀ γ", start=-1, end=2, step=0.1, value=np.log10(gamma), width=150
)
log_k_slider = bokeh.models.Slider(
    title="log₁₀ k", start=-1, end=2, step=0.1, value=np.log10(k), width=150
)
n_slider = bokeh.models.Slider(
    title="n", start=0.1, end=10, step=0.1, value=2, width=150
)
log_ks_slider = bokeh.models.Slider(
    title="log₁₀ kₛ", start=-2, end=2, step=0.1, value=np.log10(ks), width=150
)
ns_slider = bokeh.models.Slider(
    title="nₛ", start=0.1, end=10, step=0.1, value=10, width=150
)
t0_slider = bokeh.models.Slider(
    title="t₀", start=0.01, end=10, step=0.01, value=4.0, width=150
)
tau_slider = bokeh.models.Slider(
    title="τ", start=0.01, end=10, step=0.01, value=2.0, width=150
)

We also want to be able to toggle between display of normalized versus unnormalized responses. We can make a toggle button for that.

In [34]:
normalize_toggle = bokeh.models.Toggle(label='Normalize', active=True, width=50)

Finally, the legend may occasionally get in the way, so we want to toggle its visibility.

In [35]:
legend_toggle = bokeh.models.Toggle(label='Legend', active=True, width=50)

#### Step 3: Build the callbacks

We can now build our callback. This callback will be a bit more complicated. For each new slider value, we need to re-integrate the dynamical equations. We also need to check to see if the normalization toggle button is clicked and appropriately scale the data and change the y-axis label. We also will need to recalculate the result if the range of the time axis changes, so we need to read the time points off of the `x_range` property of the plot.

In [36]:
def neg_auto_callback(attr, old, new):
    # Set up time values, keeping minimum at zero
    t = np.linspace(0, p.x_range.end, 2000)

    # Package slider values
    s_args = (t0_slider.value, tau_slider.value)
    args = (
        10 ** log_beta0_slider.value,
        10 ** log_gamma_slider.value,
        10 ** log_k_slider.value,
        n_slider.value,
        10 ** log_ks_slider.value,
        ns_slider.value,
        s_pulse,
        s_args,
    )
    args_unreg = (
        10 ** log_beta0_slider.value,
        10 ** log_gamma_slider.value,
        10 ** log_ks_slider.value,
        ns_slider.value,
        s_pulse,
        s_args,
    )

    # Integrate ODES
    x = scipy.integrate.odeint(neg_auto_rhs_s_fun, x0, t, args=args)
    x = x.transpose()[0]
    x_unreg = scipy.integrate.odeint(unreg_rhs_s_fun, x0, t, args=args_unreg)
    x_unreg = x_unreg.transpose()[0]

    # Also calculate the input
    s = s_pulse(t, *s_args)

    # Normalize if desired
    if normalize_toggle.active:
        if x.max() > 0:
            x /= x.max()
        if x_unreg.max() > 0:
            x_unreg /= x_unreg.max()
        p.yaxis.axis_label = "normalized concentration"
    else:
        p.yaxis.axis_label = "concentration"

    # Show or hide legend
    if legend_toggle.active:
        p.legend.visible = True
    else:
        p.legend.visible = False
        
    # Update data source
    cds.data = dict(t=t, x=x, s=s, x_unreg=x_unreg)

Now we link the callback to the sliders, and also to the normalization toggle and the range of the time axis.

In [37]:
if fully_interactive_plots:
    log_beta0_slider.on_change("value", neg_auto_callback)
    log_gamma_slider.on_change("value", neg_auto_callback)
    log_k_slider.on_change("value", neg_auto_callback)
    n_slider.on_change("value", neg_auto_callback)
    log_ks_slider.on_change("value", neg_auto_callback)
    ns_slider.on_change("value", neg_auto_callback)
    t0_slider.on_change("value", neg_auto_callback)
    tau_slider.on_change("value", neg_auto_callback)
    normalize_toggle.on_change("active", neg_auto_callback)
    legend_toggle.on_change("active", neg_auto_callback)
    p.x_range.on_change("end", neg_auto_callback)
else:
    log_beta0_slider.disabled = True
    log_gamma_slider.disabled = True
    log_k_slider.disabled = True
    n_slider.disabled = True
    log_ks_slider.disabled = True
    ns_slider.disabled = True
    t0_slider.disabled = True
    tau_slider.disabled = True
    normalize_toggle.disabled = True
    legend_toggle.disabled = True

#### Step 4: The layout

We can now lay things out. I will put the sliders and normalization toggle in a column next to the plot.

In [38]:
layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=30),
    bokeh.layouts.column(
        log_beta0_slider,
        log_gamma_slider,
        log_k_slider,
        n_slider,
        legend_toggle,
    ),
    bokeh.layouts.column(
        log_ks_slider,
        ns_slider,
        t0_slider,
        tau_slider,
        normalize_toggle,
    )
)

#### Step 5: The app

And now for the app! (Note again that this app will not appear in the HTML rendering of this notebook and will not work using Google Colab.)

In [40]:
def app(doc):
    doc.add_root(layout)

if fully_interactive_plots:
    bokeh.io.show(app, notebook_url=notebook_url)
else:
    bokeh.io.show(layout)

By moving around the sliders, we can see that increasing the strength of the repression (by decreasing $k$) accentuates the speed-up provided by negative autoregulation. Furthermore, we see that increasing the cooperativitity of the repressor (by increasing $n$) makes the initial rise in $x$ "sharper". What other properties can you find through interacting with this plot?

## Experimental demonstration of negative autoregulation speeding response time

Can this acceleration be observed experimentally? To find out, <a href="https://doi.org/10.1016/S0022-2836(02)00994-4">Rosenfeld et al.</a> engineered a simple synthetic system based on a bacterial repressor called TetR, fused to a fluorescent protein for readout, and studied its turn-on dynamics in bacterial populations. 

<br />

<div style="width: 350px; margin: auto;">

![negative autoregulation experiment](figs/negative_autoregulation_experiment.png)

</div>


<center>This image is taken from <a href="https://doi.org/10.1016/S0022-2836(02)00994-4">Rosenfeld et al., <emph>J. Mol. Biol.</emph>, 2002</a>.</center>

<br />


Interestingly, these dynamics show the expected acceleration, as well as some oscillations around steady-state, which may be explained by time delays in the regulatory system (something we will discuss more in an upcoming chapter).

<!--
I think the below is old stuff from a previous version. Commented out for now.

WE DO NEED TO REPLACE THE ABOVE FIGURE.

To conclude this section: We now have identified another simple **design principle:** **_Negative autoregulation speeds the response time of a transcription factor_**.


Negative autoregulation can have additional functions beyond acceleration. Using similar synthetic approaches, negative autoregulation was shown to reduce stochastic cell-cell variability ("noise") in gene expression ([Becskei and Serrano, _Nature_, 2000](https://doi.org/10.1038/35014651)).

<div class="alert alert-block alert-info">


*But negative autoregulation can increase noise too, see Ghusinga et al 2017 PNAS "First-passage time approach to controlling noise in the timing of intracellular events". Worth mentioning, or too much at this point?*
    
</div>

## Summary

* Protein degradation and dilution rates determine (and limit) the switching speed of a simple transcriptinally regulated gene.
* **Design principle:** _Negative autoregulation accelerates turn-on of a transcription factor._
* **Design principle:** _Positive, ultrasensitive autoregulation generates bistability._
* Even simple circuits of 1 or 2 genes can generate interesting functional capabilities.
* Synthetic circuits can be used to test the functions of simple circuits in living cells.

This is our first foray into the analysis of dynamical systems. As we continue to work with dynamical systems, [Strogatz's book](https://www.amazon.com/gp/product/0813349109/) is a great introduction, and includes discussion on using nullclines in the analysis.
-->

## Computing environment

In [41]:
%load_ext watermark
%watermark -v -p numpy,scipy,bokeh,jupyterlab

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 8.1.1

numpy     : 1.20.3
scipy     : 1.7.3
bokeh     : 2.4.2
jupyterlab: 3.3.2



<hr>

## Problems

- [2.1: The cost of a steady state](../problems/02/problem_2.1.ipynb)
- [2.2: Event handling for discontinuous derivatives](../problems/02/problem_2.2.ipynb)